In [1]:
import requests
import pandas as pd
import json

nyc_arrests = pd.DataFrame()
temp_df = pd.DataFrame()

# 'GET' request
for offset in range(0, 230000, 50000):
    url = 'https://data.cityofnewyork.us/resource/uip8-fykc.json?$limit=50000&$offset='+str(offset)
    response = requests.get(url)
    data = response.json()
    temp_df = pd.DataFrame()
    temp_df = pd.json_normalize(data)
    
    nyc_arrests = nyc_arrests._append(temp_df, ignore_index=True)

# check number of rows and columns in the dataframe
nyc_arrests.shape

(63621, 25)

In [2]:
nyc_arrests.tail(5)

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,...,y_coord_cd,latitude,longitude,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,:@computed_region_efsh_h5xi,geocoded_column.type,geocoded_column.coordinates
63616,282877412,2024-02-27T00:00:00.000,339,"LARCENY,PETIT FROM OPEN AREAS,",341,PETIT LARCENY,PL 1552500,M,K,60,...,149165,40.5761,-73.989545,21,2,45,35,18184,Point,"[-73.989545, 40.5761]"
63617,283628088,2024-03-12T00:00:00.000,511,"CONTROLLED SUBSTANCE, POSSESSI",235,DANGEROUS DRUGS,PL 2200300,M,M,13,...,208933,40.740151,-73.994805,11,4,10,7,12074,Point,"[-73.994805, 40.740151]"
63618,283387006,2024-03-07T00:00:00.000,397,"ROBBERY,OPEN AREA UNCLASSIFIED",105,ROBBERY,PL 160102A,F,K,73,...,183789,40.671104,-73.913562,55,2,17,46,17614,Point,"[-73.913562, 40.671104]"
63619,283013910,2024-03-01T00:00:00.000,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN",348,VEHICLE AND TRAFFIC LAWS,VTL05110A4,M,K,83,...,193913,40.698893568968515,-73.91752179908735,42,2,37,53,13828,Point,"[-73.91752179908735, 40.698893568968515]"
63620,283495531,2024-03-10T00:00:00.000,203,"TRESPASS 3, CRIMINAL",352,CRIMINAL TRESPASS,PL 140100A,M,S,123,...,133050,40.531707,-74.192126,15,1,9,77,10699,Point,"[-74.192126, 40.531707]"


In [3]:
# print column names
print(nyc_arrests.columns.tolist())
nyc_arrests['geocoded_column.type'].unique()

['arrest_key', 'arrest_date', 'pd_cd', 'pd_desc', 'ky_cd', 'ofns_desc', 'law_code', 'law_cat_cd', 'arrest_boro', 'arrest_precinct', 'jurisdiction_code', 'age_group', 'perp_sex', 'perp_race', 'x_coord_cd', 'y_coord_cd', 'latitude', 'longitude', ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q', ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih', ':@computed_region_efsh_h5xi', 'geocoded_column.type', 'geocoded_column.coordinates']


array(['Point'], dtype=object)

In [4]:
# Drop redundant columns from the dataframe
nyc_arrests.drop(nyc_arrests.loc[:, ':@computed_region_f5dn_yrer':'geocoded_column.coordinates'].columns, inplace=True, axis=1)
print(nyc_arrests.columns.tolist())

['arrest_key', 'arrest_date', 'pd_cd', 'pd_desc', 'ky_cd', 'ofns_desc', 'law_code', 'law_cat_cd', 'arrest_boro', 'arrest_precinct', 'jurisdiction_code', 'age_group', 'perp_sex', 'perp_race', 'x_coord_cd', 'y_coord_cd', 'latitude', 'longitude']


In [5]:
# return indices for empty entries/values
nyc_arrests[nyc_arrests['arrest_key'] == ''].index

# check for NULL values
import numpy as np
np.where(pd.isnull(nyc_arrests['arrest_key']))

# check if all the values are numeric
pd.to_numeric(nyc_arrests['arrest_key'], errors='coerce').notnull().all()

# check if all the values are 9 digits long
nyc_arrests[nyc_arrests['arrest_key'].apply(lambda x: len(str(x)) != 9)]

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,x_coord_cd,y_coord_cd,latitude,longitude


In [6]:
# check for NULL values
import numpy as np
np.where(pd.isnull(nyc_arrests['arrest_date']))

# check if all values are datetime
nyc_arrests['arrest_date'].astype(str).apply(lambda x: pd.to_datetime(x, errors='coerce')).notna().all()

True

In [7]:
# check if all the values are numeric
pd.to_numeric(nyc_arrests['pd_cd'], errors='coerce').notnull().all()

# find unique values
nyc_arrests['pd_cd'].unique()

# find number of rows with 'nan' entries 
nyc_arrests['pd_cd'].isnull().sum()

# check if all the values are 3 digits long
temp = nyc_arrests[nyc_arrests['pd_cd'].apply(lambda x: len(str(x)) != 3)]

temp['pd_cd'].value_counts()

pd_cd
49    340
16     45
15     11
35      2
29      2
2       1
12      1
Name: count, dtype: int64

In [8]:
# find unique values
nyc_arrests['pd_desc'].unique()

array(['ROBBERY,OPEN AREA UNCLASSIFIED', 'RAPE 1', 'ASSAULT 3',
       'CONTROLLED SUBSTANCE, POSSESSI', 'ASSAULT 2,1,UNCLASSIFIED',
       'JOSTLING', '(null)', 'RAPE 2', 'STRANGULATION 1ST', 'ARSON 2,3,4',
       'BURGLARY,UNCLASSIFIED,UNKNOWN', 'ROBBERY,CAR JACKING', 'RAPE 3',
       'WEAPONS POSSESSION 3', 'RECKLESS ENDANGERMENT 1',
       'TRAFFIC,UNCLASSIFIED MISDEMEAN', 'LARCENY,PETIT FROM OPEN AREAS,',
       'TRESPASS 3, CRIMINAL', 'LARCENY,GRAND FROM PERSON,UNCL',
       'THEFT OF SERVICES, UNCLASSIFIE', 'CONTROLLED SUBSTANCE,SALE 3',
       'MENACING,UNCLASSIFIED', 'LARCENY,GRAND OF AUTO',
       'CRIMINAL MIS 2 & 3', 'LEWDNESS,PUBLIC',
       'CRIMINAL MISCHIEF,UNCLASSIFIED 4', 'OBSTR BREATH/CIRCUL',
       'TRAFFIC,UNCLASSIFIED INFRACTIO', 'PUBLIC ADMINISTATION,UNCLASS M',
       'WEAPONS, POSSESSION, ETC', 'DISORDERLY CONDUCT',
       'CHILD, ENDANGERING WELFARE', 'CRIMINAL CONTEMPT 1',
       'PUBLIC ADMINISTRATION,UNCLASSI', 'BAIL JUMPING 3',
       'ASSAULT POLICE/PEAC

In [9]:
# check if all the values are numeric
pd.to_numeric(nyc_arrests['ky_cd'], errors='coerce').notnull().all()

# find unique values
nyc_arrests['ky_cd'].unique()
nyc_arrests['ky_cd'].isnull().sum()
nyc_arrests.loc[nyc_arrests['ky_cd'].isnull()]

# check if all the values are 3 digits long
nyc_arrests[nyc_arrests['ky_cd'].apply(lambda x: len(str(x)) != 3)]

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,x_coord_cd,y_coord_cd,latitude,longitude


In [10]:
# find unique values
nyc_arrests['ofns_desc'].unique()

array(['ROBBERY', 'RAPE', 'ASSAULT 3 & RELATED OFFENSES',
       'DANGEROUS DRUGS', 'FELONY ASSAULT', 'JOSTLING', '(null)', 'ARSON',
       'BURGLARY', 'DANGEROUS WEAPONS', 'MISCELLANEOUS PENAL LAW',
       'VEHICLE AND TRAFFIC LAWS', 'PETIT LARCENY', 'CRIMINAL TRESPASS',
       'GRAND LARCENY', 'OTHER OFFENSES RELATED TO THEF',
       'GRAND LARCENY OF MOTOR VEHICLE', 'CRIMINAL MISCHIEF & RELATED OF',
       'OFF. AGNST PUB ORD SENSBLTY &', 'OTHER TRAFFIC INFRACTION',
       'OFFENSES AGAINST PUBLIC ADMINI', 'DISORDERLY CONDUCT',
       'SEX CRIMES', 'FORGERY', 'NYS LAWS-UNCLASSIFIED FELONY',
       'POSSESSION OF STOLEN PROPERTY', 'OTHER STATE LAWS',
       'MURDER & NON-NEGL. MANSLAUGHTE', 'INTOXICATED & IMPAIRED DRIVING',
       'OFFENSES INVOLVING FRAUD', 'CANNABIS RELATED OFFENSES',
       'OFFENSES AGAINST THE PERSON', 'FRAUDS', 'FOR OTHER AUTHORITIES',
       'ALCOHOLIC BEVERAGE CONTROL LAW', 'OTHER STATE LAWS (NON PENAL LA',
       'ENDAN WELFARE INCOMP', 'UNAUTHORIZED USE OF 

In [11]:
# find unique values
nyc_arrests['law_cat_cd'].unique()
nyc_arrests.loc[nyc_arrests['law_cat_cd'] == '9']
nyc_arrests.loc[nyc_arrests['law_cat_cd'] == 'I']

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,x_coord_cd,y_coord_cd,latitude,longitude
1295,282058775,2024-02-12T00:00:00.000,849,"NY STATE LAWS,UNCLASSIFIED VIO",677,OTHER STATE LAWS,LOC00000I0,I,M,30,1,25-44,M,BLACK,996945,238532,40.821381691686916,-73.95413257822179
3632,280473444,2024-01-13T00:00:00.000,922,"TRAFFIC,UNCLASSIFIED MISDEMEAN",348,VEHICLE AND TRAFFIC LAWS,VTL03750T2,I,K,78,0,25-44,M,BLACK,993703,182685,40.668101,-73.965923
4787,280063664,2024-01-06T00:00:00.000,940,LEAVING THE SCENE / PROPERTY DAMAGE / INJURED ...,881,OTHER TRAFFIC INFRACTION,VTL060001A,I,K,90,0,45-64,F,WHITE HISPANIC,998016,196598,40.706283,-73.950348
4911,279846195,2024-01-02T00:00:00.000,972,SEAT BELTS,880,MOVING INFRACTIONS,VTL1229C00,I,K,90,0,25-44,M,BLACK,999668,195748,40.703949,-73.944391
5165,280927162,2024-01-22T00:00:00.000,969,"TRAFFIC,UNCLASSIFIED INFRACTIO",881,OTHER TRAFFIC INFRACTION,VTL0511A01,I,Q,109,0,45-64,M,WHITE HISPANIC,1032852,228739,40.79437808324494,-73.82447061233147
7896,281063433,2024-01-24T00:00:00.000,968,UNLICENSED OPERATOR,880,MOVING INFRACTIONS,VTL0509001,I,K,68,0,25-44,M,WHITE HISPANIC,979616,170814,40.63552029862381,-74.01669615060825
8919,280854796,2024-01-21T00:00:00.000,969,"TRAFFIC,UNCLASSIFIED INFRACTIO",881,OTHER TRAFFIC INFRACTION,VTL112800A,I,Q,112,0,25-44,F,WHITE HISPANIC,1029052,203660,40.725562860059284,-73.83836166699767
10356,281494723,2024-02-01T00:00:00.000,973,USE CELL PHONE WHILE DRIVING,880,MOVING INFRACTIONS,VTL1225C2A,I,K,66,0,25-44,M,WHITE,985588,169978,40.63322676726574,-73.99517939541789
11274,280209808,2024-01-09T00:00:00.000,940,LEAVING THE SCENE / PROPERTY DAMAGE / INJURED ...,881,OTHER TRAFFIC INFRACTION,VTL060001A,I,M,13,0,25-44,M,BLACK HISPANIC,988971,207813,40.737074,-73.982962
11420,280041976,2024-01-05T00:00:00.000,969,"TRAFFIC,UNCLASSIFIED INFRACTIO",881,OTHER TRAFFIC INFRACTION,VTL040201A,I,K,66,0,18-24,M,WHITE HISPANIC,984900,171853,40.638373320496456,-73.99765797112055


In [12]:
# find unique values
nyc_arrests['arrest_boro'].unique()

array(['M', 'K', 'Q', 'B', 'S'], dtype=object)

In [44]:
# check if all the values are numeric
pd.to_numeric(nyc_arrests['arrest_precinct'], errors='coerce').notnull().all()

True

In [45]:
# check if all the values are numeric
pd.to_numeric(nyc_arrests['jurisdiction_code'], errors='coerce').notnull().all()

True

In [46]:
# find unique values
nyc_arrests['age_group'].unique()

array(['<18', '25-44', '65+', '45-64', '18-24'], dtype=object)

In [47]:
# find unique values
nyc_arrests['perp_sex'].unique()

array(['M', 'F'], dtype=object)

In [48]:
# find unique values
nyc_arrests['perp_race'].unique()

array(['BLACK', 'WHITE HISPANIC', 'WHITE', 'ASIAN / PACIFIC ISLANDER',
       'UNKNOWN', 'BLACK HISPANIC', 'AMERICAN INDIAN/ALASKAN NATIVE'],
      dtype=object)

In [16]:
# calculate min & max on 'latitude'
print(nyc_arrests['latitude'].agg(['min', 'max']))

# calculate min & max on 'longitude'
print(nyc_arrests['longitude'].agg(['min', 'max']))

# locate rows with longitude value equal to 0
nyc_arrests.loc[nyc_arrests['longitude'] == '0.0']

#locate rows with latitude value equal to 0
nyc_arrests.loc[nyc_arrests['latitude'] == '0.0']

temp=nyc_arrests.drop([41238,40573])

print(temp['longitude'].agg(['min', 'max']))
print(temp['latitude'].agg(['min', 'max']))

min          0.0
max    40.911236
Name: latitude, dtype: object
min    -73.700717
max           0.0
Name: longitude, dtype: object
min           -73.700717
max    -74.2527114132383
Name: longitude, dtype: object
min    40.501092
max    40.911236
Name: latitude, dtype: object


In [13]:
#load the data into sql server using replace option
!pip install pyodbc
import sqlalchemy as sal
engine=sal.create_engine('mssql://RahityaGovindu/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn=engine.connect()

In [14]:
#load data into sql using append option
nyc_arrests.to_sql('nyc_arrests_sql',con=conn,index=False,if_exists='append')

53